<a href="https://colab.research.google.com/github/sp7412/colab/blob/master/feature_extractor_resnet_v2_50.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!nvidia-smi --query-gpu=gpu_name,driver_version,memory.total --format=csv

name, driver_version, memory.total [MiB]
Tesla K80, 418.67, 11441 MiB


In [4]:
!nvidia-smi

Mon Jul 27 15:09:09 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.51.05    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   54C    P8    33W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

import tensorflow_hub as hub
import tensorflow_datasets as tfds

from tensorflow.keras import layers
import logging
logger = tf.get_logger()
logger.setLevel(logging.ERROR)

In [ ]:
(training_set, validation_set), dataset_info = tfds.load(
    'tf_flowers',
    split=['train[:70%]', 'train[70%:]'],
    with_info=True,
    as_supervised=True,
)

In [ ]:
num_classes = dataset_info.features['label'].num_classes

num_training_examples = 0
num_validation_examples = 0

for example in training_set:
  num_training_examples += 1

for example in validation_set:
  num_validation_examples += 1

print('Total Number of Classes: {}'.format(num_classes))
print('Total Number of Training Images: {}'.format(num_training_examples))
print('Total Number of Validation Images: {} \n'.format(num_validation_examples))

Total Number of Classes: 5
Total Number of Training Images: 2569
Total Number of Validation Images: 1101 



In [ ]:
for i, example in enumerate(training_set.take(5)):
  print('Image {} shape: {} label: {}'.format(i+1, example[0].shape, example[1]))

Image 1 shape: (333, 500, 3) label: 2
Image 2 shape: (212, 320, 3) label: 3
Image 3 shape: (240, 320, 3) label: 3
Image 4 shape: (240, 320, 3) label: 4
Image 5 shape: (317, 500, 3) label: 3


In [ ]:
IMAGE_RES = 224

def format_image(image, label):
  image = tf.image.resize(image, (IMAGE_RES, IMAGE_RES))/255.0
  return image, label

BATCH_SIZE = 32

train_batches = training_set.shuffle(num_training_examples//4).map(format_image).batch(BATCH_SIZE).prefetch(1)

validation_batches = validation_set.map(format_image).batch(BATCH_SIZE).prefetch(1)

In [ ]:
model = tf.keras.Sequential([
    hub.KerasLayer("https://tfhub.dev/google/imagenet/resnet_v2_50/feature_vector/4",
                   trainable=False),  # Can be True, see below.
    tf.keras.layers.Dense(num_classes, activation='softmax')
])
model.build([None, 224, 224, 3])  # Batch input shape.

In [ ]:
URL = "https://tfhub.dev/google/imagenet/resnet_v2_50/feature_vector/4"
feature_extractor = hub.KerasLayer(URL,
                                   input_shape=(IMAGE_RES, IMAGE_RES, 3),
                                   trainable=True, arguments=dict(batch_norm_momentum=0.997))
model.build([None, 224, 224, 3])  # Batch input shape.

In [ ]:
feature_extractor.trainable = False

In [ ]:
model = tf.keras.Sequential([
  feature_extractor,
  layers.Dense(num_classes)
])

model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer_3 (KerasLayer)   (None, 2048)              23564800  
_________________________________________________________________
dense_3 (Dense)              (None, 5)                 10245     
Total params: 23,575,045
Trainable params: 23,529,605
Non-trainable params: 45,440
_________________________________________________________________


In [ ]:
model.compile(
  optimizer='adam',
  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['accuracy'])

EPOCHS = 1000

history = model.fit(train_batches,
                    epochs=EPOCHS,
                    validation_data=validation_batches)

Epoch 1/1000
81/81 [==============================] - 16s 194ms/step - loss: 0.4174 - accuracy: 0.9774 - val_loss: 147.1828 - val_accuracy: 0.3152
Epoch 2/1000
81/81 [==============================] - 15s 186ms/step - loss: 0.5329 - accuracy: 0.9591 - val_loss: 22.3212 - val_accuracy: 0.4532
Epoch 3/1000
81/81 [==============================] - 15s 187ms/step - loss: 0.4529 - accuracy: 0.9833 - val_loss: 4.5332 - val_accuracy: 0.6476
Epoch 4/1000
81/81 [==============================] - 15s 188ms/step - loss: 0.3951 - accuracy: 0.9922 - val_loss: 1.7841 - val_accuracy: 0.7611
Epoch 5/1000
81/81 [==============================] - 15s 188ms/step - loss: 0.3577 - accuracy: 0.9961 - val_loss: 1.1719 - val_accuracy: 0.8038
Epoch 6/1000
81/81 [==============================] - 15s 186ms/step - loss: 0.4117 - accuracy: 0.9833 - val_loss: 1.6162 - val_accuracy: 0.7975
Epoch 7/1000
81/81 [==============================] - 15s 188ms/step - loss: 0.5411 - accuracy: 0.9607 - val_loss: 1.1676 - val

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
saved_model_path = "/content/drive/My Drive/20200723-213702resnet_simclr_model.h5"
feature_extractor = tf.keras.models.load_model(saved_model_path)

In [ ]:
for idx, layer in enumerate(feature_extractor.layers):
    layer.trainable = False

In [ ]:
feature_extractor.layers[-1].trainable = True
# feature_extractor.layers[-2].trainable = True

In [ ]:
feature_extractor.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
resnet50 (Model)             (None, 7, 7, 2048)        23587712  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 256)               524544    
_________________________________________________________________
activation (Activation)      (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               32896     
_________________________________________________________________
activation_1 (Activation)    (None, 128)               0     

In [ ]:
output_layer = layers.Dense(num_classes, activation='softmax', name='scores')(feature_extractor.output)

In [ ]:
model = tf.keras.Model(feature_extractor.input, output_layer)

In [ ]:
model.compile(
  optimizer='adam',
  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['accuracy'])

In [ ]:
for layer in feature_extractor.layers:
  print(layer.trainable)

False
False
False
False
False
False
False
True


In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
resnet50 (Model)             (None, 7, 7, 2048)        23587712  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 256)               524544    
_________________________________________________________________
activation (Activation)      (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               32896     
_________________________________________________________________
activation_1 (Activation)    (None, 128)               0     

In [ ]:
EPOCHS = 20

history = model.fit(train_batches,
                    epochs=EPOCHS,
                    validation_data=validation_batches)

Epoch 1/20
81/81 [==============================] - 8s 95ms/step - loss: 1.5599 - accuracy: 0.3223 - val_loss: 1.5449 - val_accuracy: 0.3488
Epoch 2/20
81/81 [==============================] - 7s 83ms/step - loss: 1.5223 - accuracy: 0.3663 - val_loss: 1.5397 - val_accuracy: 0.3451
Epoch 3/20
81/81 [==============================] - 7s 84ms/step - loss: 1.5073 - accuracy: 0.3873 - val_loss: 1.5374 - val_accuracy: 0.3442
Epoch 4/20
81/81 [==============================] - 7s 83ms/step - loss: 1.5069 - accuracy: 0.3858 - val_loss: 1.5355 - val_accuracy: 0.3560
Epoch 5/20
81/81 [==============================] - 7s 83ms/step - loss: 1.5019 - accuracy: 0.3947 - val_loss: 1.5295 - val_accuracy: 0.3624
Epoch 6/20
81/81 [==============================] - 7s 82ms/step - loss: 1.5033 - accuracy: 0.3916 - val_loss: 1.5285 - val_accuracy: 0.3678
Epoch 7/20
81/81 [==============================] - 7s 84ms/step - loss: 1.5003 - accuracy: 0.3928 - val_loss: 1.5268 - val_accuracy: 0.3615
Epoch 8/20
81